# 1. Build your own convolutional neural network using pytorch

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

In [ ]:
# ============ CONFIG - Update these paths for your environment ============
DATA_ROOT = os.path.join(os.getcwd(), "data")  # Or set absolute path: "/path/to/Cowtestdata"
TRAIN_PATH = os.path.join(DATA_ROOT, "Train")
TEST_PATH = os.path.join(DATA_ROOT, "Test")
OUTPUT_DIR = os.path.join(os.getcwd(), "outputs")
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(512 * 7 * 7, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 4)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = self.pool(F.relu(self.conv5(x)))
        x = x.view(-1, 512 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 2. Train your model using cow teat datasets (you may need to use  Google Colab (or Kaggle) with GPU to train your code) 

### (1) use torchvision.datasets.ImageFolder for the training dataset
### (2) use custom dataloader for test dataset (return image tensor and file name)

In [3]:
class CustomTestDataset(Dataset):
    """Custom test dataset that returns image tensor, label, and file name."""
    IMG_EXTENSIONS = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff'}

    def __init__(self, root_dir, transforms=None):
        self.root_dir = root_dir
        self.file_names = sorted([f for f in os.listdir(root_dir)
                                  if os.path.splitext(f)[1].lower() in self.IMG_EXTENSIONS])
        self.transforms = transforms

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = os.path.join(self.root_dir, file_name)
        image = Image.open(file_path).convert('RGB')
        if self.transforms:
            image = self.transforms(image)
        # Parse label from filename (e.g. c1_xxx.jpg -> 1)
        label = int(file_name.split("_")[0][1:])
        return image, label, file_name

In [4]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define the transforms for the test dataset
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [5]:
# Define the training and test datasets
train_dataset = torchvision.datasets.ImageFolder(root=TRAIN_PATH, transform=train_transform)
test_dataset = CustomTestDataset(root_dir=TEST_PATH, transforms=test_transform)

# Define the dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 3. Evaluate your model using the developed software

In [6]:
# Reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True

# Device (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Initialize the model and optimizer
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Training loop
loss_values = []
for epoch in range(10):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    epoch_loss = running_loss / len(train_dataloader)
    loss_values.append(epoch_loss)
    print("Epoch %d, loss: %.3f" % (epoch + 1, epoch_loss))

Epoch 1, loss: 1.139
Epoch 2, loss: 0.970
Epoch 3, loss: 0.932
Epoch 4, loss: 0.917
Epoch 5, loss: 0.902
Epoch 6, loss: 0.899
Epoch 7, loss: 0.897
Epoch 8, loss: 0.884
Epoch 9, loss: 0.859
Epoch 10, loss: 0.853


In [7]:
# Plot training loss curve
plt.figure(figsize=(8, 5))
plt.plot(loss_values, 'b-o', linewidth=2, markersize=6)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss over Epochs')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, 'training_loss.png'), dpi=150, bbox_inches='tight')
plt.show()

# Evaluation
predicted_values = []
all_filenames = []
all_labels = []
correct = 0
total = 0

model.eval()
with torch.no_grad():
    for images, labels, filenames in test_dataloader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted.cpu() == labels).sum().item()
        predicted_values.extend(predicted.cpu().tolist())
        all_filenames.extend(filenames)
        all_labels.extend(labels.tolist())

In [8]:
# Accuracy and per-class metrics
accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

# Confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(all_labels, predicted_values)
print("\nConfusion Matrix:")
print(cm)
print("\nClassification Report:")
class_names = train_dataset.classes if hasattr(train_dataset, 'classes') else ['class_0', 'class_1', 'class_2', 'class_3']
print(classification_report(all_labels, predicted_values, target_names=class_names))

# Save predictions CSV
df = pd.DataFrame({'Test File': all_filenames, 'True Label': all_labels, 'Predicted': predicted_values})
df.to_csv(os.path.join(OUTPUT_DIR, 'predictions.csv'), index=False)


In [10]:
torch.save(model.state_dict(), os.path.join(OUTPUT_DIR, 'model.pt'))
print(f"Model saved to {OUTPUT_DIR}/model.pt")

![project.PNG](attachment:project.PNG)

# 4. Compare results with [SCTL paper](https://www.mdpi.com/2076-2615/12/7/886/htm). Requirement: performance is better than VGG16: 66.8%

We could achive only 61.842 %

# 5. Write a four-page paper report using the shared LaTex template. Upload your paper to ResearchGate or Arxiv, and put your paper link here.

https://github.com/bhansalijainam/Neural-project/blob/main/Image%20classification%20using%20CNN%20with%20pytorch.pdf

Uploaded on github after review will upload on research gate

# 6. Grading rubric

(1). Code ------- 20 points (you also need to upload your final model as a pt file)

(2). Grammer ---- 20 points

(3). Introduction & related work --- 10 points


(4). Method  ---- 20 points

(5). Results ---- 20 points

     > = 66.8% -->10 points
     < 40 % -->0 points
     >= 40 % & < 66.8% --> 0.3731 point/percent
     

(6). Discussion - 10 points

https://drive.google.com/file/d/1vFQYmbQCun5k5K5LNLWxjNQuKT-i2ObU/view?usp=sharing

this is the link of my weight shared over here due to file size